In [ ]:
import os
import json
import yaml

from IPython.display import display, Markdown

import numpy as np
import pandas as pd

# from decode_trf import delivery_data_from_logfile

from pymedphys.msq import mosaiq_connect
from pymedphys.utilities import get_index, get_data_directory, get_filepath, get_gantry_tolerance
from pymedphys.logfile import (
    compare_logfile_group_bygantry
)

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)

In [ ]:
data_directory = get_data_directory(config)

cache_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison.yaml')
cache_scratch_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison_scratch.yaml')

with open(cache_filepath, 'r') as cache_file:
    cache = yaml.load(cache_file)

In [ ]:
patient_ids = list(cache.keys())
len(patient_ids)

In [ ]:
data = []

for patient_id in patient_ids:
    for byfield in cache[patient_id]:
        for angle in cache[patient_id][byfield]:
            comparison = cache[patient_id][byfield][angle]['median']
            file_hashes = cache[patient_id][byfield][angle]['median_filehash_group']
            all_comparisons = cache[patient_id][byfield][angle]['comparisons']
            all_comparisons_flat = []
            for key, value in all_comparisons.items():
                all_comparisons_flat.append(value)
            
            data.append([
                patient_id, byfield, angle, comparison, file_hashes, tuple(all_comparisons_flat)
            ])
            
comparisons_table = pd.DataFrame(
    columns=[
        'patient_id', 'field_id',
        'gantry_angle', 'comparison',
        'file_hashes', 'all_comparisons'
    ],
    data=data
)

comparisons_table = comparisons_table.sort_values('comparison', ascending=False)
top_ten = comparisons_table.iloc[0:10]
top_ten

In [ ]:
len(comparisons_table)

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    for row in top_ten.itertuples():
        display(Markdown(
            "### Patient ID: {}\n\n"
            "Field ID: {}\n\n"
            "Gantry Angle: {}\n\n"
            "Cached Median Comparison: {}\n\n"
            "All Comparisons: {}".format(
                row.patient_id,
                row.field_id,
                row.gantry_angle,
                row.comparison,
                row.all_comparisons
            )
        ))
        logfile_group = row.file_hashes
        gantry_angle = row.gantry_angle
        
        comparison = compare_logfile_group_bygantry(
            index, config, cursor, logfile_group, gantry_angle)